In [1]:
import sys
sys.path.append('..')

In [2]:
from core.dataset import *
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns

In [3]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [4]:
dataset = Data(load_dir, ligands, sheet_type, merge=True, numpy=False)

In [5]:
cleaned_data = dataset.fillna(0)

In [6]:
cleaned_data.isna().sum().sum()

0

In [7]:
cleaned_data.shape

(14376, 985)

In [8]:
cleaned_data.iloc[:, 984]

0       0
1       0
2       0
3       0
4       0
       ..
1597    7
1598    7
1599    7
1600    7
1601    7
Name: 0, Length: 14376, dtype: int32

In [9]:
X = cleaned_data.iloc[:, [i for i in range(984)]]
y = cleaned_data.iloc[:, 984]

In [10]:
X.isna().sum().sum()

0

In [11]:
y.isna().sum().sum()

0

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
model = LogisticRegression()

### for logistic regression:
* need to interpolate/remove NaNs

In [15]:
model.fit(X_train, y_train)

C:\Users\minha\miniconda3\envs\peep\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
pred = model.predict(X_val)

In [17]:
y_val = np.array(y_val)

In [18]:
count = 0
for y_pred, y_true in zip(pred, y_val):
    if y_pred == y_true:
        count += 1
print(count/len(y_val))

0.5104311543810849


In [19]:
model.coef_.shape

(8, 984)

### 8 classes, different set of coefficients for each one, how to generalize them to one set of feature importance?

In [114]:
coefficients = []
for i in model.coef_:
    cdf = pd.DataFrame(i, X.columns, columns=['Coefficients'])
    coefficients.append(cdf)

In [115]:
coefficients[0]

,Coefficients
time_series_1,0.000426
time_series_2,-0.002988
time_series_3,-0.008587
time_series_4,-0.004772
time_series_5,-0.000211
...,...
fold_change_96,0.000578
fold_change_97,0.000636
fold_change_98,0.000595
max_fold_change,0.000550


In [116]:
sorted_CpG_coefficients = coefficients[0].sort_values(by=['Coefficients'])

In [117]:
sorted_CpG_coefficients

,Coefficients
derivatives_1,-0.048896
ipt_1,-0.015126
derivatives_13,-0.012656
derivatives_14,-0.012223
derivatives_12,-0.010847
...,...
derivatives_7,0.013674
derivatives_8,0.015789
derivatives_5,0.021471
derivatives_4,0.039600


In [118]:
most_important_features = sorted_CpG_coefficients[984 - 7:]

In [119]:
cleaned_data[most_important_features.index]

,valley_times_2,pk1_width,derivatives_7,derivatives_8,derivatives_5,derivatives_4,derivatives_3
0,25,6.000000,3.672079,6.661474,2.719529,0.000000,-1.372320
1,34,3.000000,6.175272,7.407052,-0.894742,-0.894742,-0.297270
2,18,7.000000,4.101602,4.051103,0.300717,0.300717,-0.229325
3,25,3.000000,-1.059758,-3.091504,-2.726745,0.000000,10.026322
4,33,7.000000,6.134909,9.178994,3.697769,1.299437,0.677397
...,...,...,...,...,...,...,...
1597,21,4.000000,-0.638243,-2.572907,-1.896395,-1.436777,-2.647025
1598,20,3.000000,-5.926792,-8.389690,14.079013,1.006600,0.000000
1599,18,4.000000,0.000000,0.415806,0.466846,-1.262564,0.000000
1600,19,5.885976,-11.737464,-14.030435,13.468512,9.344059,-0.431694


### Fit sklearn and xgboost model with these 7 features, test accuracy?

In [120]:
df_most_important_features = cleaned_data[most_important_features.index]
df_most_important_features = pd.concat([df_most_important_features, cleaned_data.iloc[:, [984]]], axis=1)

In [121]:
df_most_important_features

,valley_times_2,pk1_width,derivatives_7,derivatives_8,derivatives_5,derivatives_4,derivatives_3,0
0,25,6.000000,3.672079,6.661474,2.719529,0.000000,-1.372320,0
1,34,3.000000,6.175272,7.407052,-0.894742,-0.894742,-0.297270,0
2,18,7.000000,4.101602,4.051103,0.300717,0.300717,-0.229325,0
3,25,3.000000,-1.059758,-3.091504,-2.726745,0.000000,10.026322,0
4,33,7.000000,6.134909,9.178994,3.697769,1.299437,0.677397,0
...,...,...,...,...,...,...,...,...
1597,21,4.000000,-0.638243,-2.572907,-1.896395,-1.436777,-2.647025,7
1598,20,3.000000,-5.926792,-8.389690,14.079013,1.006600,0.000000,7
1599,18,4.000000,0.000000,0.415806,0.466846,-1.262564,0.000000,7
1600,19,5.885976,-11.737464,-14.030435,13.468512,9.344059,-0.431694,7


In [122]:
X2, y2 = df_most_important_features.iloc[:, [i for i in range(7)]], df_most_important_features.iloc[:, [7]]

In [123]:
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, y2, test_size=0.1, random_state=42)

In [125]:
y2_train = y2_train.to_numpy()
y2_train = y2_train.reshape((-1, ))

In [127]:
model2 = LogisticRegression()

In [128]:
model2.fit(X2_train, y2_train)

C:\Users\minha\miniconda3\envs\peep\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [129]:
pred = model2.predict(X2_val)

#### log reg model does not entirely agree with xgboost ensemble model, will look into this deeper

In [130]:
count = 0
for y_pred, y_true in zip(pred, y2_val[0]):
    if y_pred == y_true:
        count += 1
print(count / len(y2_val))

0.3337969401947149


In [131]:
import xgboost as xgb

In [132]:
model3 = xgb.XGBClassifier()

In [133]:
model3.fit(X2_train, y2_train)

C:\Users\minha\miniconda3\envs\peep\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:54:03] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [134]:
pred = model3.predict(X2_val)

In [135]:
count = 0
for y_pred, y_true in zip(pred, y2_val[0]):
    if y_pred == y_true:
        count += 1
print(count / len(y2_val))

0.40681502086230875


### now try fitting log reg and xgboost on xgboost selected features

In [154]:
extracted_features = [dataset['integrals_pos_38'], dataset['min_trough2peak'], dataset['integrals_pos_11'], dataset['integrals_40'],
                      dataset['integrals_pos_14'], dataset['time_series_2'], dataset['integrals_pos_12']]

In [155]:
data = pd.DataFrame()

In [156]:
data = pd.concat(extracted_features, axis=1)
labels = dataset.iloc[:, [984]]

In [157]:
cleaned_data = data.fillna(0)

In [139]:
data = data.to_numpy()
labels = labels.to_numpy()

In [140]:
model4 = xgb.XGBClassifier()

In [141]:
X3_train, X3_val, y3_train, y3_val = train_test_split(data, labels, test_size=0.1, random_state=42)

In [145]:
y3_train, y3_val = y3_train.reshape((-1, )), y3_val.reshape((-1, ))

In [146]:
model4.fit(X3_train, y3_train)

[14:55:01] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [147]:
pred = model4.predict(X3_val)

In [150]:
count = 0
for y_pred, y_true in zip(pred, y3_val):
    if y_pred == y_true:
        count += 1
print(count / len(y2_val))

0.480528511821975


### LogReg

In [151]:
model5 = LogisticRegression()

In [162]:
cleaned_data = cleaned_data.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [163]:
labels = labels.to_numpy()

In [164]:
X4_train, X4_val, y4_train, y4_val = train_test_split(cleaned_data, labels, test_size=0.1, random_state=42)

In [166]:
y4_train = y4_train.reshape((-1, ))
y4_val = y4_val.reshape((-1, ))

In [167]:
model5.fit(X4_train, y4_train)

C:\Users\minha\miniconda3\envs\peep\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [168]:
pred = model5.predict(X4_val)

In [171]:
count = 0
for y_pred, y_true in zip(pred, y4_val):
    if y_pred == y_true:
        count += 1
print(count / len(y4_val))

0.4478442280945758


## Log Reg trained on XGBoost features are much more accurate... only ~6% accuracy difference

* 